In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import when, col
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import *


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
import sys
#!{sys.executable} -m pip install pmdarima

In [4]:
sc = pyspark.SparkContext(appName="6242-project")
sqlContext = SQLContext(sc)

F:\ProgramData\conda\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
## loading by zip data
#df = sqlContext.read.format("csv").option("header", True).option("sep", "\t").option("inferSchema", "true").csv("zip_code_market_tracker.tsv000")
df = sqlContext.read.format("csv").option("header", True).option("sep", "\t").option("inferSchema", "true").csv("C:/Users/shawn/Documents/hw3/gJu81ErL5Q_hw3/hw3-skeleton/Q1/zip_code_market_tracker/zip_code_market_tracker.tsv000")
##loading byregion data
#df = sqlContext.read.format("csv").option("header", True).option("sep", "\t").option("inferSchema", "true").csv("weekly_housing_market_data_most_recent.tsv000")

In [7]:
# df_null = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns])
# df_null.show()

In [6]:
#load zip code - city matching data:
df_zip = sqlContext.read.format("csv").option("header", True).option("sep", ",").option("inferSchema", "true").csv("zip_code_database_cleaned.csv")

# Preprocessing: by zip data analysis:

In [9]:
# df.printSchema()

In [10]:
# df.select('period_duration').distinct().show()

In [11]:
# df.select("period_begin").distinct().repartition(1).write.csv('distinct period begin')

In [7]:
df_period_begin = df.select("period_begin").distinct()

In [13]:
# df.select('property_type').distinct().show()

In [8]:
df = df.withColumn('zip_code',col('region').substr(11,15).alias("zip_code"))

In [15]:
# df.printSchema()

In [16]:
# df_zip.printSchema()

In [9]:
from pyspark.sql.functions import lpad
df_zip = df_zip.withColumn('zip',lpad(df_zip.zip,5,'0'))

In [10]:
tb = df.join(df_zip,df.zip_code == df_zip.zip,"left")\
.select('zip','zip_code','city','primary_city',df_zip.state,'county','state_code','latitude','longitude','property_type','median_sale_price','homes_sold','period_begin','period_end')

In [19]:
# tb_null = tb.select([count(when(isnull(c), c)).alias(c) for c in tb.columns])
# tb_null.show()

In [20]:
#calculate the weighted average median sold price in every city

In [21]:
# tb.show()

In [22]:
# tb.printSchema()

Fill in Null Value 

In [11]:
tb = tb.orderBy("period_begin")

In [12]:
df_period = tb.select("period_begin").distinct().orderBy("period_begin")

In [25]:
# df_period.show()

In [13]:
#get unique zip code
df_zip = tb.select("zip").distinct()

In [14]:
period_list = list(df_period.select('period_begin').toPandas()['period_begin'])

In [15]:
df_zip=df_zip.withColumn("period_begin", F.array([F.lit(x) for x in period_list]))

In [16]:
df_zip_period = df_zip.select("zip", F.explode("period_begin")).withColumnRenamed("col", "period_begin")

In [30]:
#test with one zip code
# df_zip_period.filter(df_zip_period.zip==22031).orderBy("period_begin").show()

In [17]:
#left join and fill in null with previous value
w = Window.partitionBy("zip").orderBy("period_begin")

tb = df_zip_period.join(tb, ["zip", "period_begin"], "left").select(
    "zip",
    "period_begin",
    *[F.last(F.col(c), ignorenulls=True).over(w).alias(c)
      for c in tb.columns if c not in ("zip", "period_begin")
     ]
).drop(tb.zip).drop(tb.period_begin)

In [32]:
# probably there are issues when the first period is null. Need to investigate later 
# tb.show()

In [33]:

# tb_null = tb.select([count(when(isnull(c), c)).alias(c) for c in tb.columns])

In [34]:
# tb_null.show(20)

In [18]:
#'primary_city','county','state')
weighted_city_sales = tb.groupBy('property_type','period_begin','period_end','county','state')\
.agg(sum(col('median_sale_price')*col('homes_sold'))/sum(col('homes_sold')))\
.withColumnRenamed("(sum((median_sale_price * homes_sold)) / sum(homes_sold))","weighted_average_median_sale_price")\
.withColumn("weighted_average_median_sale_price",col("weighted_average_median_sale_price").cast('decimal(36,5)'))

In [36]:
# weighted_city_sales.show()

In [19]:
weighted_city_sales = weighted_city_sales.filter(weighted_city_sales.property_type.isNotNull())

In [38]:
# weighted_city_sales= weighted_city_sales.withColumn("weighted_average_median_sale_price", F.last("weighted_average_median_sale_price", True).over(Window.partitionBy(col("state"), col("primary_city"),col("property_type")).orderBy("period_begin").rowsBetween(-sys.maxsize,0)))

In [20]:
weighted_city_sales = weighted_city_sales.na.fill(value=0, subset=['weighted_average_median_sale_price'])

In [21]:
#check if there are null values in weighted_city_sales 

df_null = weighted_city_sales.select([count(when(isnull(c), c)).alias(c) for c in weighted_city_sales.columns])

In [22]:
df_null.show()

+-------------+------------+----------+------+-----+----------------------------------+
|property_type|period_begin|period_end|county|state|weighted_average_median_sale_price|
+-------------+------------+----------+------+-----+----------------------------------+
|            0|           0|         0|     0|    0|                                 0|
+-------------+------------+----------+------+-----+----------------------------------+



In [42]:
# weighted_city_sales.filter("county is NULL").coalesce(1).write.csv('null county.csv')


In [43]:
# weighted_city_sales.filter((weighted_city_sales.primary_city == "Victor")).show(126)

In [44]:
# weighted_city_sales.filter("weighted_average_median_sale_price is NULL").show(126)

In [45]:
# weighted_city_sales.filter((weighted_city_sales.primary_city == "Aguadilla")).show()


## ARIMA:

In [48]:
from pmdarima.arima import auto_arima
import pandas as pd

In [49]:
def city_forecast(tb,n_periods=36):
    forecast = pd.DataFrame(columns = ['county','state','2023','2024','2025'] )
    pred = {}
    
    print(tb['county'].iloc[0],'     ',tb['state'].iloc[0])
    
    #for property_type in tb['property_type'].unique().tolist():
        #print(property_type)
        #if property_type == 'Multi-Family (2-4 Unit)':
        #    property_type = 'Multi-Family'
        
    data = tb.loc[:,['period_begin','weighted_average_median_sale_price']]
    data.reset_index(drop=True,inplace=True)

    data['period_begin'] = data["period_begin"].astype('datetime64[ns]')

    #if data['period_begin'][0] > data['period_begin'][2]:
    #    data = data.iloc[::-1]
    data.sort_values(by = ['period_begin'],inplace=True)
    data = data.set_index('period_begin')

    model = auto_arima(data,m = 12,seasonal = True)

    pred['pred'],pred['conf'] = model.predict(n_periods=n_periods, return_conf_int=True)
    d = pd.DataFrame(pred['pred'],columns = ['price'])

    forecast = forecast.append({'county':tb['county'].iloc[0],'state':tb['state'].iloc[0],'2023':d.iloc[(6)][0],'2024':d.iloc[(6+12)][0],'2025':d.iloc[(6+12+12)][0]},ignore_index=True)

    return forecast

In [46]:
city_pair = weighted_city_sales.toPandas()

In [47]:
## get the distinct city-county-state pair
county_pair = weighted_city_sales.select('county','state').distinct().toPandas()

In [6]:
print(city_pair.shape)
print(county_pair.shape)

NameError: name 'city_pair' is not defined

In [31]:
d = city_pair.loc[(city_pair['county']=='Fairfax County') & (city_pair['state']=='VA') &(city_pair['property_type']=='All Residential'),['property_type','period_begin','county','state','weighted_average_median_sale_price']]

city_forecast(tb=d)

Fairfax County       VA
0


F:\ProgramData\conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


,county,state,2023,2024,2025
0,Fairfax County,VA,690365.017483,674723.335845,697623.312605


In [1]:
df =pd.DataFrame(columns = ['county','state','2023','2024','2025'] )
df = df.append(city_forecast(tb= d))
df.head()

NameError: name 'pd' is not defined

In [132]:
#df =pd.DataFrame(columns = ['county','state','2023','2024','2025'] )
i=1755
county_pair1 = county_pair.iloc[i:1756,:]

for county,state in zip(county_pair1['county'],county_pair1['state']):
    if county is not None:
        print(i)
        tb = city_pair.loc[(city_pair['county']==county) & (city_pair['state']==state) & (city_pair['property_type']=='All Residential'),['property_type','period_begin','county','state','weighted_average_median_sale_price']]
        df = df.append(city_forecast(tb= tb))
        i+=1

In [131]:
#skip 295
#Dunklin County       MO
#
df.to_csv('forecast_part3_1607_1755.csv')

In [135]:
county_pair.iloc[-2,:]

county    Caledonia County
state                   VT
Name: 1753, dtype: object

In [134]:
df.tail()

,county,state,2023,2024,2025
0,Fayette County,AL,547846.084828,695792.169655,843738.254483
0,Alpine County,CA,501738.061735,501751.276637,501751.276637
0,Mason County,IL,86349.790218,86233.585516,86233.582638
0,Hopewell city,VA,247778.204718,318693.236784,389610.136724
0,Caledonia County,VT,218599.854321,151419.483303,178259.504931


## Merge CSVs together:

In [1]:
import pandas as pd
import glob
path = r"C:\Users\shawn\OneDrive - Georgia Institute of Technology\CSE 6242\Project\Real Estate Data\forecast result\*.csv"
df_total =pd.DataFrame(columns = ['county','state','2023','2024','2025'] )
for fname in glob.glob(path):
    df_total=df_total.append(pd.read_csv(fname))
    print(len(df_total))

100
395
440
481
554
570
599
631
632
756
856
861
867
884
896
914
940
979
1103
1207
1285
1406
1437
1444
1490
1603
1660
1698
1705


In [4]:
df_total.head()

,county,state,2023,2024,2025,Unnamed: 0
0,Osage County,MO,162043.458661,162043.458661,162043.458661,0.0
1,Laclede County,MO,228352.518634,228352.518634,228352.518634,0.0
2,Pottawatomie County,KS,168978.777614,157329.021179,158797.026005,0.0
3,Fresno County,CA,287902.051934,284880.634946,284834.120074,0.0
4,Rockland County,NY,445910.225041,445910.225041,445910.225041,0.0


# Append back missing state/county pair:

In [ ]:
## Merge Previous 10 years with forecast 3 years of data
## Use June House Price (All Residential type)

In [ ]:
#calculate state average, and national average as 2 new columns:

In [ ]:
## Any county have wired data, will fill with state average: (such as large sales price and negative price)

In [ ]:
## link our table to the full county table

In [ ]:
## Any County Missing data, will fill them with state average: we have about 2000 unique county

In [ ]:
## Any state missing data, will fill them with national average:
## output to csv file:
df_total.to_csv('forecast_total.csv')

# Merge historical and forecast together:

## Process the historical data to take the July of each year for all property type:

In [23]:
weighted_city_sales.show()

+--------------------+-------------------+-------------------+-----------------+-----+----------------------------------+
|       property_type|       period_begin|         period_end|           county|state|weighted_average_median_sale_price|
+--------------------+-------------------+-------------------+-----------------+-----+----------------------------------+
|           Townhouse|2019-08-01 00:00:00|2019-10-31 00:00:00|   Hampden County|   MA|                      185723.68421|
|Single Family Res...|2017-09-01 00:00:00|2017-11-30 00:00:00|   Hampden County|   MA|                      213451.04722|
|Single Family Res...|2014-05-01 00:00:00|2014-07-31 00:00:00| Worcester County|   MA|                      235172.26563|
|Single Family Res...|2017-03-01 00:00:00|2017-05-31 00:00:00| Worcester County|   MA|                      245898.65798|
|Multi-Family (2-4...|2020-09-01 00:00:00|2020-11-30 00:00:00| Worcester County|   MA|                      366801.47059|
|Single Family Res...|20

In [25]:
weighted_city_sales.dtypes

[('property_type', 'string'),
 ('period_begin', 'timestamp'),
 ('period_end', 'timestamp'),
 ('county', 'string'),
 ('state', 'string'),
 ('weighted_average_median_sale_price', 'decimal(36,5)')]

In [36]:
historical = weighted_city_sales.filter((col('property_type')=='All Residential')&(month(col('period_begin'))==7))\
.select('period_begin',year('period_begin').alias('year'),'county','state','weighted_average_median_sale_price')

In [35]:
historical.select('county','state').count() ##how many county has july sales?

1743

In [37]:
historical.show()

+-------------------+----+-----------------+-----+----------------------------------+
|       period_begin|year|           county|state|weighted_average_median_sale_price|
+-------------------+----+-----------------+-----+----------------------------------+
|2019-07-01 00:00:00|2019|   Windsor County|   VT|                      249240.45304|
|2019-07-01 00:00:00|2019|Chittenden County|   VT|                      353232.62840|
|2017-07-01 00:00:00|2017| Jefferson County|   NY|                      133420.69825|
|2020-07-01 00:00:00|2020| Jefferson County|   NY|                      163269.38272|
|2014-07-01 00:00:00|2014|   Chester County|   PA|                     1620000.00000|
|2018-07-01 00:00:00|2018|New Castle County|   DE|                      175000.00000|
|2012-07-01 00:00:00|2012|Rockingham County|   VA|                      173179.72222|
|2016-07-01 00:00:00|2016|       Lee County|   NC|                      172329.34132|
|2019-07-01 00:00:00|2019|  Alamance County|   NC|    

In [39]:
historical_df=historical.select('year','state','county','weighted_average_median_sale_price')\
            .groupBy('state','county')\
            .pivot('year')\
            .avg('weighted_average_median_sale_price')\
            .toPandas()

In [46]:
historical_df.head()

,state,county,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,WI,Oconto County,None,None,None,135816.949150000,141953.225810000,160271.186440000,166615.384620000,177849.295770000,151282.456140000,207724.257430000,239000.000000000
1,AR,Greene County,115272.727270000,75000.000000000,117187.109380000,60744.725740000,52666.666667000,36000.000000000,70233.333330000,165000.000000000,108473.809523000,86843.750000000,196000.000000000
2,FL,Monroe County,383000.000000000,347500.000000000,598750.000000000,372500.000000000,576914.649680000,502985.294120000,567514.792900000,665530.681820000,676250.000000000,685267.960605000,876193.195265000
3,OR,Morrow County,87000.000000000,158500.000000000,218333.333335000,59000.000000000,None,110000.000000000,146750.000000000,93184.782610000,235714.285710000,238565.000000000,279740.272730000
4,GA,Murray County,None,None,None,93562.500000000,62307.000000000,None,158000.000000000,318300.000000000,172500.000000000,None,228995.000000000


In [53]:
for col in historical_df.columns[2:]:
    historical_df[col] = pd.to_numeric(historical_df[col],errors = 'coerce').fillna(0).round()
historical_df.head()

,state,county,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,WI,Oconto County,0.0,0.0,0.0,135817.0,141953.0,160271.0,166615.0,177849.0,151282.0,207724.0,239000.0
1,AR,Greene County,115273.0,75000.0,117187.0,60745.0,52667.0,36000.0,70233.0,165000.0,108474.0,86844.0,196000.0
2,FL,Monroe County,383000.0,347500.0,598750.0,372500.0,576915.0,502985.0,567515.0,665531.0,676250.0,685268.0,876193.0
3,OR,Morrow County,87000.0,158500.0,218333.0,59000.0,0.0,110000.0,146750.0,93185.0,235714.0,238565.0,279740.0
4,GA,Murray County,0.0,0.0,0.0,93562.0,62307.0,0.0,158000.0,318300.0,172500.0,0.0,228995.0


## Merge historical with forecast (forecast left join historical)

In [55]:
forecast_df = pd.read_excel(r'forecast_total_with function.xlsx')

In [56]:
merge_df = pd.merge(forecast_df,historical_df,how='outer',on=['state','county'])

In [58]:
merge_df

,county,state,2023,2024,2025,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Osage County,MO,162043.4587,162043.4587,162043.4587,0.0,0.0,0.0,0.0,0.0,39000.0,39000.0,119626.0,107450.0,73300.0,172250.0
1,Laclede County,MO,228352.5186,228352.5186,228352.5186,73591.0,93933.0,106203.0,77712.0,90500.0,89833.0,0.0,189212.0,179125.0,115000.0,230085.0
2,Pottawatomie County,KS,168978.7776,157329.0212,158797.0260,0.0,0.0,0.0,0.0,0.0,123500.0,191750.0,139600.0,116312.0,176554.0,125483.0
3,Fresno County,CA,287902.0519,284880.6349,284834.1201,168464.0,218917.0,131903.0,246207.0,140168.0,230928.0,233152.0,214071.0,214647.0,310576.0,282700.0
4,Rockland County,NY,445910.2250,445910.2250,445910.2250,0.0,0.0,0.0,375118.0,364210.0,460851.0,427219.0,481882.0,475211.0,567345.0,531562.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1747,Moore County,TX,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205000.0
1748,Oldham County,TX,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,219900.0
1749,Val Verde County,TX,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,189950.0
1750,Keokuk County,IA,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50500.0,235000.0,0.0


In [59]:
historical_df.shape

(1743, 13)

In [60]:
forecast_df.shape

(1744, 5)

In [68]:
col = merge_df.columns.tolist()
col = col[1:2] + col[0:1] + col[5:] + col[2:5]
merge_df=merge_df[col]


Index([ 'state', 'county',   '2012',   '2013',   '2014',   '2015',   '2016',
         '2017',   '2018',   '2019',   '2020',   '2021',   '2022',     2023,
           2024,     2025],
      dtype='object')

In [80]:
merge_df.groupby(['state','county']).nunique().reset_index().sort_values(by='2020',ascending=False)

,state,county,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,AK,Anchorage Municipality,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1164,OH,Muskingum County,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1175,OH,Stark County,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1174,OH,Shelby County,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1173,OH,Seneca County,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,ID,Jefferson County,0,0,0,0,0,0,0,0,0,0,0,1,1,1
266,GA,Clay County,0,0,0,0,0,0,0,0,0,0,0,1,1,1
383,ID,Blaine County,0,0,0,0,0,0,0,0,0,0,0,1,1,1
1035,NM,Chaves County,0,0,0,0,0,0,0,0,0,0,0,1,1,1


In [71]:
merge_df.to_csv('merge_total.csv')

In [1]:
import pandas as pd
import numpy as np

In [14]:
merge_df= pd.read_csv(r'merge_total.csv').iloc[:,1:]

In [15]:
merge_df.head()

,state,county,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,AK,Anchorage Municipality,245603.0,347093.0,263103.0,371592.0,328279.0,279300.0,318783.0,274215.0,360335.0,371997.0,357995.0,384813.0216,383959.9844,383959.5618
1,AK,city-Wasilla,0.0,0.0,0.0,0.0,0.0,0.0,0.0,235000.0,0.0,0.0,0.0,337066.6667,383466.6667,429866.6667
2,AK,Fairbanks North Star Borough,0.0,228750.0,123000.0,164917.0,227370.0,250875.0,84750.0,211125.0,160261.0,209750.0,359900.0,315766.1055,315505.5953,315505.6256
3,AK,Matanuska-Susitna Borough,167825.0,222505.0,182803.0,159910.0,142610.0,275149.0,182349.0,278242.0,298748.0,341652.0,268529.0,313390.5216,311082.7793,311067.3633
4,AK,Municipality of Anchorage,237213.0,311167.0,490000.0,442750.0,475500.0,485000.0,0.0,536767.0,520450.0,450275.0,433675.0,462276.8044,462276.8044,462276.8044


In [16]:
merge_df.iloc[1, :] = merge_df.replace(0, np.nan).bfill(axis = 1).iloc[1, :]

In [28]:
for row in range(merge_df.shape[0]):
    merge_df.iloc[row, :] = merge_df.replace(0, np.nan).bfill(axis = 1).iloc[row, :]
    merge_df.iloc[row, :] = merge_df.replace(0, np.nan).ffill(axis = 1).iloc[row, :]

In [21]:
merge_df.head()

,state,county,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,AK,Anchorage Municipality,245603.0,347093.0,263103.0,371592.0,328279.0,279300.0,318783.0,274215.0,360335.0000,371997.0000,357995.0000,384813.0216,383959.9844,383959.5618
1,AK,city-Wasilla,235000.0,235000.0,235000.0,235000.0,235000.0,235000.0,235000.0,235000.0,337066.6667,337066.6667,337066.6667,337066.6667,383466.6667,429866.6667
2,AK,Fairbanks North Star Borough,228750.0,228750.0,123000.0,164917.0,227370.0,250875.0,84750.0,211125.0,160261.0000,209750.0000,359900.0000,315766.1055,315505.5953,315505.6256
3,AK,Matanuska-Susitna Borough,167825.0,222505.0,182803.0,159910.0,142610.0,275149.0,182349.0,278242.0,298748.0000,341652.0000,268529.0000,313390.5216,311082.7793,311067.3633
4,AK,Municipality of Anchorage,237213.0,311167.0,490000.0,442750.0,475500.0,485000.0,536767.0,536767.0,520450.0000,450275.0000,433675.0000,462276.8044,462276.8044,462276.8044


In [29]:
merge_df.isnull().values.any()

False

In [30]:
pd.isnull(merge_df).any(1).to_numpy().nonzero()[0]

array([], dtype=int64)

In [32]:
merge_df.iloc[ 896,:]

state                MS
county    Monroe County
2012           106060.0
2013           106060.0
2014           106060.0
2015           106060.0
2016           106060.0
2017           106060.0
2018           106060.0
2019           106060.0
2020           106060.0
2021           106060.0
2022           106060.0
2023           106060.0
2024           106060.0
2025           106060.0
Name: 896, dtype: object

In [31]:
merge_df.to_csv('merge_total_2.csv')

# Calculate Ratio:

In [21]:
df = pd.read_csv(r'C:\Users\shawn\OneDrive - Georgia Institute of Technology\CSE 6242\Project\Tableau Input\Total_final_allgender.csv')

In [22]:
df.head()

,state,county,House Price 2012,House Price 2013,House Price 2014,House Price 2015,House Price 2016,House Price 2017,House Price 2018,House Price 2019,...,2016_Ratio,2017_Ratio,2018_Ratio,2019_Ratio,2020_Ratio,2021_Ratio,2022_Ratio,2023_Ratio,2024_Ratio,2025_Ratio
0,AR,Franklin County,72000.0,72000.0,29880.0,51627.0,79940.0,176750.0,160167.0,241750.0,...,3.013874,6.757790,5.967696,8.680119,6.497884,6.096643,8.673434,8.462511,8.261834,8.070234
1,AR,Washington County,105682.0,93740.0,139978.0,175105.0,177520.0,182420.0,178695.0,112687.0,...,6.609084,6.516629,5.996275,3.728025,9.821954,11.416734,10.523400,12.030908,11.626763,11.432261
2,NC,Vance County,85000.0,85000.0,85000.0,85000.0,85000.0,102359.0,130133.0,165490.0,...,3.563792,4.074963,5.309601,6.411111,4.854151,4.707033,6.433959,5.883353,6.450134,6.865022
3,PA,Clinton County,95042.0,142683.0,70500.0,92060.0,101260.0,111000.0,166944.0,74833.0,...,4.139989,4.363722,6.156660,2.540242,4.158992,3.148683,5.182187,4.032445,3.969477,3.873502
4,OR,Polk County,310000.0,159000.0,208032.0,206000.0,276641.0,262015.0,298750.0,318058.0,...,10.304738,9.627241,10.579341,10.457618,8.528110,9.187881,11.959770,10.581501,10.364399,10.192951


In [23]:
df.columns

Index(['state', 'county', 'House Price 2012', 'House Price 2013',
       'House Price 2014', 'House Price 2015', 'House Price 2016',
       'House Price 2017', 'House Price 2018', 'House Price 2019',
       'House Price 2020', 'House Price 2021', 'House Price 2022',
       'House Price 2023', 'House Price 2024', 'House Price 2025',
       'Gender_2012', 'Gender_2013', 'Gender_2014', 'Gender_2015',
       'Gender_2016', 'Gender_2017', 'Gender_2018', 'Gender_2019',
       'Gender_2020', 'Gender_2021', 'Gender_2022', 'Gender_2023',
       'Gender_2024', 'Gender_2025', 'Gender', '2012_Ratio', '2013_Ratio',
       '2014_Ratio', '2015_Ratio', '2016_Ratio', '2017_Ratio', '2018_Ratio',
       '2019_Ratio', '2020_Ratio', '2021_Ratio', '2022_Ratio', '2023_Ratio',
       '2024_Ratio', '2025_Ratio'],
      dtype='object')

In [24]:
df_male = df.loc[df['Gender'] == 'male',['state', 'county', 
       'Gender_2012', 'Gender_2013', 'Gender_2014', 'Gender_2015',
       'Gender_2016', 'Gender_2017', 'Gender_2018', 'Gender_2019',
       'Gender_2020', 'Gender_2021', 'Gender_2022', 'Gender_2023',
       'Gender_2024', 'Gender_2025']]
df_female = df.loc[df['Gender'] == 'female',['state', 'county', 
       'Gender_2012', 'Gender_2013', 'Gender_2014', 'Gender_2015',
       'Gender_2016', 'Gender_2017', 'Gender_2018', 'Gender_2019',
       'Gender_2020', 'Gender_2021', 'Gender_2022', 'Gender_2023',
       'Gender_2024', 'Gender_2025']]

In [25]:
df_female= df_female.set_index(['state', 'county'])
df_male = df_male.set_index(['state', 'county'])
df_male.head()

Gender_2012  Gender_2013  Gender_2014  Gender_2015  \
state county                                                                  
AR    Franklin County          28378        27750        32262        32247   
      Washington County        28457        28070        29462        29766   
NC    Vance County             27185        26826        27201        24153   
PA    Clinton County           30729        31753        31978        31922   
OR    Polk County              34535        32328        31461        31055   

                         Gender_2016  Gender_2017  Gender_2018  Gender_2019  \
state county                                                                  
AR    Franklin County          32121        31528        32428        31192   
      Washington County        30470        31513        32478        32200   
NC    Vance County             23861        26036        26400        27216   
PA    Clinton County           31496        32288        33563        35185   
OR    Polk County              31333        31936        33857        36912   

                         Gender_2020  Gender_2021  Gender_2022  Gender_2023  \
state county                                                                  
AR    Franklin County          36528        37128        37728        38328   
      Washington County        32216        32740        33265        33789   
NC    Vance County             47653        48475        49298        50121   
PA    Clinton County           36671        37259        37847        38435   
OR    Polk County              43255        43733        44212        44691   

                         Gender_2024  Gender_2025  
state county                                       
AR    Franklin County          38928        39528  
      Washington County        34314        34839  
NC    Vance County             50944        51766  
PA    Clinton County           39023        39611  
OR    Polk County              45170        45649

In [26]:
df_female.head()

Gender_2012  Gender_2013  Gender_2014  Gender_2015  \
state county                                                                  
AR    Franklin County          17708        18063        20158        20604   
      Washington County        21182        21487        21709        22927   
NC    Vance County             22324        24079        25565        23625   
PA    Clinton County           16319        16660        17487        18514   
OR    Polk County              21477        21876        22557        22215   

                         Gender_2016  Gender_2017  Gender_2018  Gender_2019  \
state county                                                                  
AR    Franklin County          21687        22282        23122        24077   
      Washington County        23498        24713        25684        26337   
NC    Vance County             23841        24228        21953        24409   
PA    Clinton County           19730        20280        21585        22942   
OR    Polk County              23100        23093        23238        24765   

                         Gender_2020  Gender_2021  Gender_2022  Gender_2023  \
state county                                                                  
AR    Franklin County          29385        30473        31561        32649   
      Washington County        19965        20361        20757        21154   
NC    Vance County             31236        31816        32396        32977   
PA    Clinton County           24729        25620        26512        27404   
OR    Polk County              34821        35712        36604        37496   

                         Gender_2024  Gender_2025  
state county                                       
AR    Franklin County          33738        34826  
      Washington County        21550        21947  
NC    Vance County             33557        34138  
PA    Clinton County           28296        29188  
OR    Polk County              38388        39280

In [27]:
df_male.sub(df_female, fill_value=0)

Gender_2012  Gender_2013  Gender_2014  Gender_2015  \
state county                                                                  
AR    Franklin County          10670         9687        12104        11643   
      Washington County         7275         6583         7753         6839   
NC    Vance County              4861         2747         1636          528   
PA    Clinton County           14410        15093        14491        13408   
OR    Polk County              13058        10452         8904         8840   
...                              ...          ...          ...          ...   
IL    Tazewell County          16720        17071        15996        16085   
NY    Cayuga County            10965        10815        11121        11127   
TN    Jefferson County          6130         6107         8233         8935   
WI    Waupaca County           13917        13598         8601        10056   
MO    Jefferson County         11358        10692        10938        11757   

                         Gender_2016  Gender_2017  Gender_2018  Gender_2019  \
state county                                                                  
AR    Franklin County          10434         9246         9306         7115   
      Washington County         6972         6800         6794         5863   
NC    Vance County                20         1808         4447         2807   
PA    Clinton County           11766        12008        11978        12243   
OR    Polk County               8233         8843        10619        12147   
...                              ...          ...          ...          ...   
IL    Tazewell County          16013        17470        16636        17269   
NY    Cayuga County            10481        11501        12127        11153   
TN    Jefferson County          9194         9439        10946        11301   
WI    Waupaca County           12763        13652        15085        14849   
MO    Jefferson County         12222        14191        13821        14239   

                         Gender_2020  Gender_2021  Gender_2022  Gender_2023  \
state county                                                                  
AR    Franklin County           7143         6655         6167         5679   
      Washington County        12251        12379        12508        12635   
NC    Vance County             16417        16659        16902        17144   
PA    Clinton County           11942        11639        11335        11031   
OR    Polk County               8434         8021         7608         7195   
...                              ...          ...          ...          ...   
IL    Tazewell County          12117        11857        11598        11339   
NY    Cayuga County            10322        10243        10164        10085   
TN    Jefferson County          8532         8903         9275         9646   
WI    Waupaca County           17611        17910        18210        18509   
MO    Jefferson County         10820        10862        10904        10945   

                         Gender_2024  Gender_2025  
state county                                       
AR    Franklin County           5190         4702  
      Washington County        12764        12892  
NC    Vance County             17387        17628  
PA    Clinton County           10727        10423  
OR    Polk County               6782         6369  
...                              ...          ...  
IL    Tazewell County          11080        10820  
NY    Cayuga County            10005         9926  
TN    Jefferson County         10018        10389  
WI    Waupaca County           18809        19108  
MO    Jefferson County         10987        11028  

[1729 rows x 14 columns]

In [28]:
df_precent_wage_diff = df_male.sub(df_female, fill_value=0).div(df_male, fill_value = 0) # % diff of wage between male and female.

In [31]:
df_precent_wage_diff = df_precent_wage_diff.reset_index()

In [38]:
df_final = pd.merge(df, df_precent_wage_diff, how='inner', on=['state','county'],suffixes=('', '_diff'))

In [39]:
df_final.to_csv('Total_final_allgender_with_diff.csv')